In [1]:
import sys, datetime
sys.path.append("../../scripts/")
from s3_support import *

import pandas as pd
import numpy as np

# transauction

In [20]:
# get auction trans
q = '''select 
            auct.*
        from transauction as auct 
        left join transactions as t 
            on auct.transaction=t.id
        where t.status='A' '''
trans_auct = redshift_query_read(q, schema='production')

In [21]:
trans_auct.tail(3)

,id,form,transaction,bidder,product,checknumber,offlinetype,originalamount,tax,total,quantity,unitcost,status,createddate,lastupdate
1333059,58427,987393,15861806,165052,176057,0,0,225.0,0.0,225.0,1,225.0,1,2022-05-17 14:57:01,2022-05-18 16:25:21
1333060,59371,988117,15910965,166951,183701,0,0,500.0,0.0,500.0,1,500.0,1,2022-05-23 12:17:20,2022-05-23 17:54:56
1333061,59715,986452,15911333,167698,170025,0,0,21.0,0.0,21.0,3,7.0,1,2022-05-23 14:22:23,2022-05-23 19:45:25


In [22]:
"{:,}".format(len(trans_auct)), trans_auct['createddate'].min(), trans_auct['createddate'].max()

('1,333,062',
 Timestamp('2019-09-06 16:00:05'),
 Timestamp('2022-05-27 13:29:41'))

In [31]:
agg_trans = trans_auct.groupby('transaction').last()

agg_forms = agg_trans.groupby('form')['total'].agg(['count', 'sum']).reset_index()
agg_forms.tail(3)

,form,count,sum
843,988117,15,2055.0
844,988142,7,6750.0
845,988268,4,610.0


In [32]:
agg_forms.sort_values('sum', ascending=False, inplace=True)
agg_forms.head(10)

,form,count,sum
716,982705,274,64460.0
701,982334,42,62200.0
672,980633,7,62000.0
767,984712,31,59325.0
743,983653,55,57300.0
420,971807,40,55020.0
584,977431,72,48295.0
358,969913,143,47635.0
538,976128,84,47175.0
494,974904,320,47175.0


# transactions by form type

In [33]:
# form.type = 5
q = '''select 
            t.*
        from transactions as t
        left join form as f
            on t.form=f.id
        where t.status='A' and f.type=5'''
trans = redshift_query_read(q, schema='production')

In [34]:
trans.tail(3)

,id,org,form,status,amount,donations_amt,purchases_amt,events_amt,registrations_amt,events_tickets,...,smspledge_count,smspledge_amt,auctionpurchase_count,auctiondonation_count,is_fraud,channel,useragent,gift_assist_count,gift_assist_amt,qgiv_fee
123955,15947172,448301,985968,A,154.5,150.0,0.0,0.0,0.0,0,...,0,0.0,0,0,False,0,Mozilla50iPhoneCPUiPhoneOS155likeMacOSXAppleWe...,1,4.5,7.95
123956,15947876,448559,987717,A,0.0,0.0,0.0,0.0,0.0,1,...,0,0.0,0,0,False,0,None,0,0.0,0.00
123957,15949663,447848,987477,A,390.0,0.0,0.0,390.0,0.0,1,...,0,0.0,0,0,False,0,None,0,0.0,15.71


In [41]:
agg_obj = {'amount': ['count', 'sum'], 'date': ['min', 'max']}
trans.groupby('form')[['amount', 'date']].agg(agg_obj).reset_index().tail(3)

form amount                date           
              count      sum        min        max
1041  988635      2    47.25 2022-05-23 2022-05-23
1042  988683     15  4700.00 2022-05-25 2022-05-28
1043  988709      7  6409.60 2022-05-20 2022-05-27

In [44]:
agg_obj = {'amount': ['count', 'sum'], 'date': ['min', 'max']}
agg_form_trans = trans.groupby('form')[['amount', 'date']].agg(agg_obj).reset_index()
agg_form_trans.columns = ['form', 'trans_count', 'trans_sum', 'date_min', 'date_max']
agg_form_trans.sort_values('trans_sum', ascending=False, inplace=True)
agg_form_trans['duration'] = agg_form_trans['date_max'] - agg_form_trans['date_min']
agg_form_trans[['form', 'trans_count', 'trans_sum', 'duration']].head(10)

,form,trans_count,trans_sum,duration
831,982334,203,261014.04,117 days
356,967287,1000,257955.91,56 days
593,974935,232,244788.77,77 days
798,980633,71,212000.00,154 days
890,983836,248,203304.99,74 days
30,950704,328,199736.50,160 days
643,976128,270,189893.25,76 days
501,971807,238,175060.44,155 days
880,983582,154,172385.00,67 days
937,984977,1219,170298.92,65 days


In [47]:
forms = [982334, 967287, 974935, 980633, 983836,
        950704, 976128, 971807, 983582, 984977]
trans[trans['form'].isin(forms)][['form', 'org']].groupby('org').first().reset_index()

,org,form
0,442934,950704
1,444359,983836
2,445546,983582
3,446070,982334
4,446401,984977
5,446518,967287
6,446545,974935
7,446989,971807
8,447423,976128
9,447806,980633
